In [1]:
from pymongo import MongoClient
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

In [2]:
client = MongoClient()

In [3]:
def clean_itinerary(itinerary):
    def select_cols(item, cols):
        if type(cols) == list:
            tpl = [select_cols(item, k) for k in cols]
            return dict(tpl)
        elif type(cols) == dict:
            key, val = list(cols.items())[0]
            return select_cols(item[key][0], val)

        return cols, item[cols]

    keys_of_interest = {
        'Itineraries': ['OutboundLegId', {'PricingOptions': 'Price'}]
    }

    clean = {}
    clean['_id'] = itinerary['_id']
    for key, columns in keys_of_interest.items():
        clean[key] = [select_cols(item, columns) for item in itinerary[key]]

    return clean

In [4]:
db = client.skyscanner
itineraries = db.itineraries

cleaned_itineraries = db.cleaned_itineraries

In [6]:
iterator = itineraries.find()

start = 0
iterator.skip(start * 1000)

if start == 0:
    cleaned_itineraries.drop()

for i in tqdm(range(start * 1000, itineraries.count(), 1000)):
    end = min(1000, itineraries.count() - i)
    cleaned_itineraries.insert_many([clean_itinerary(next(iterator)) for item in tqdm(range(end), leave=False)])